In [2]:
import numpy as np
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
print("torch.cuda.is_available()   =", torch.cuda.is_available())
print("torch.cuda.device_count()   =", torch.cuda.device_count())
print("torch.cuda.device('cuda')   =", torch.cuda.device('cuda'))
print("torch.cuda.current_device() =", torch.cuda.current_device())

torch.cuda.is_available()   = True
torch.cuda.device_count()   = 1
torch.cuda.device('cuda')   = <torch.cuda.device object at 0x00000067A3F34F60>
torch.cuda.current_device() = 0


In [4]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import Resize, ToTensor, Normalize, Compose, Grayscale
from torch.utils.data import DataLoader
batch_size = 1
root_dir = 'data/train'

target_size = (100,100)
transforms = Compose([
                    Grayscale(num_output_channels=1),
                   # Resize(target_size), # Resizes image
                    ToTensor(),           # Converts to Tensor, scales to [0, 1] float (from [0, 255] int)
                    Normalize((0.5,), (0.5,)), # scales to [-1.0, 1.0]
                    ])

train_dataset = ImageFolder(root_dir, transform=transforms)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# Same for validation dataset
val_root_dir = 'data/val'
val_dataset = ImageFolder(val_root_dir, transform=transforms)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [6]:
# Same for test dataset
test_root_dir = 'data/test'
test_dataset = ImageFolder(test_root_dir, transform=transforms)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
print(train_dataset[0][0].shape)

torch.Size([1, 216, 216])


In [88]:
import torch.nn as nn
    
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv = nn.Sequential(
            # size: 1x216x216
            nn.Conv2d(1, 64, kernel_size=(5,1), stride= (2,1)),
            nn.LeakyReLU(0.2),
            # size: 64x106x216
            
            nn.MaxPool2d(kernel_size = (1,5), stride = 2),
            nn.LeakyReLU(0.2),
            #size: 64x106X53
            
        )
        
        self.fc = nn.Sequential(
            nn.Linear(64*106*53,10),
            nn.Softmax()
        )
    
    def forward(self, input):
        output = self.conv(input)
        output = output.view(output.size(0), 64*106*53)
        output = self.fc(output)
        return output

In [92]:
# ADDING EARLY STOPPING
import numpy as np
import torch
from copy import deepcopy

def train(model, train_loader, optimizer, loss_fn, print_every=100):
    '''
    Trains the model for one epoch
    '''
    model.train()
    losses = []
    n_correct = 0
    for iteration, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        optimizer.zero_grad()
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
        #if iteration % print_every == 0:
        #    print('Training iteration {}: loss {:.4f}'.format(iteration, loss.item()))
        losses.append(loss.item())
        n_correct += torch.sum(output.argmax(1) == labels).item()
    accuracy = 100.0 * n_correct / len(train_loader.dataset)
    return np.mean(np.array(losses)), accuracy
            
def test(model, test_loader, loss_fn):
    '''
    Tests the model on data from test_loader
    '''
    model.eval()
    test_loss = 0
    n_correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            loss = loss_fn(output, labels)
            test_loss += loss.item()
            n_correct += torch.sum(output.argmax(1) == labels).item()

    average_loss = test_loss / len(test_loader)
    accuracy = 100.0 * n_correct / len(test_loader.dataset)
    print('Test average loss: {:.4f}, accuracy: {:.3f}'.format(average_loss, accuracy))
    return average_loss, accuracy


def fit(train_dataloader, val_dataloader, model, optimizer, loss_fn, n_epochs, scheduler=None):
    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []
    best_val_loss = np.inf
    val_accuracy_best = 0
    best_model = None
    patience = 5 # if no improvement after 5 epochs, stop training
    counter = 0
    for epoch in range(n_epochs):
        train_loss, train_accuracy = train(model, train_dataloader, optimizer, loss_fn)
        val_loss, val_accuracy = test(model, val_dataloader, loss_fn)
        
        torch.cuda.empty_cache()
        
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        if scheduler:
            scheduler.step() # argument only needed for ReduceLROnPlateau
        print('Epoch {}/{}: train_loss: {:.4f}, train_accuracy: {:.4f}, val_loss: {:.4f}, val_accuracy: {:.4f}'.format(epoch+1, n_epochs,
                                                                                                          train_losses[-1],
                                                                                                          train_accuracies[-1],
                                                                                                          val_losses[-1],
                                                                                                          val_accuracies[-1]))
        ### Early stopping code
        if val_accuracy > val_accuracy_best:
            best_val_loss = val_loss
            val_accuracy_best = val_accuracy
            best_model = deepcopy(model)
            counter = 0
        else:
            counter += 1
        if counter == patience:
            print('No improvement for {} epochs; training stopped.'.format(patience))
            break
    
    return best_val_loss, val_accuracy_best

In [93]:
model_conv = ConvNet()
model_conv = model_conv.to(device)
learning_rate = 0.001
optimizer = torch.optim.Adam(model_conv.parameters(), lr=learning_rate, weight_decay=0.004)
n_epochs = 25
loss_fn = nn.CrossEntropyLoss()

In [94]:
val_loss, val_accuracy = fit(train_dataloader, val_dataloader, model_conv, optimizer, loss_fn, n_epochs)

Test average loss: 2.3609, accuracy: 10.000
Epoch 1/25: train_loss: 1.9249, train_accuracy: 53.6490, val_loss: 2.3609, val_accuracy: 10.0000
Test average loss: 2.3611, accuracy: 10.000
Epoch 2/25: train_loss: 1.7350, train_accuracy: 72.5905, val_loss: 2.3611, val_accuracy: 10.0000


KeyboardInterrupt: 